In [181]:
import mapclassify as mc
import numpy as np
from sklearn import preprocessing
from giddy.markov import Spatial_Markov,LISA_Markov,prais,Markov
import numpy.linalg as la
import libpysal as ps
import giddy
from giddy.ergodic import fmpt,  steady_state

In [517]:
def fmpt_test(p):
    p = np.asarray(p)
    k = p.shape[0]
    fmpt_all = np.zeros((k,k))
    for desti in range(k):
        b = np.ones(k-1)
        p_sub = np.delete(np.delete(p, desti, 0), desti,1)
        p_calc = np.eye(k-1) - p_sub
        m = np.full(k-1, np.inf)
        print(p_sub)
        row0 = (p_calc!=0).sum(axis=1)
        none0 = np.arange(k-1)
        try:
            m[none0] = np.linalg.solve(p_calc, b) 
        except np.linalg.LinAlgError as err:
            if 'Singular matrix' in str(err):
                if (row0==0).sum() > 0:
                    index0 = set(np.argwhere(row0 == 0).flatten())
                    x = (p_calc[:,list(index0)]!= 0).sum(axis=1) 
                    setx = set(np.argwhere(x).flatten())
                    while not setx.issubset(index0):
                        index0 = index0.union(setx)
                        x = (p_calc[:,list(index0)]!= 0).sum(axis=1) 
                        setx = set(np.argwhere(x).flatten())
                    none0 = np.asarray(list(set(none0).difference(index0)))
                    if len(none0) >= 1:  
                        p_calc = p_calc[none0,:][:, none0]
                        b = b[none0]  
                        m[none0] = np.linalg.solve(p_calc, b) 
        
        recc = np.nan_to_num((np.delete(p, desti, 1)[desti] * m), 0, posinf=np.inf).sum() + 1
        fmpt_all[:, desti] = np.insert(m, desti, recc)
    
    return fmpt_all

In [518]:
P = np.array([[1. , 0. , 0. , 0. , 0. ],
              [0. , 1. , 0. , 0. , 0. ],
              [0. , 0. , 0. , 0.5, 0.5],
              [0. , 0.5, 0. , 0. , 0.5],
              [0.5, 0. , 0. , 0.5, 0. ]])

In [519]:
fmpt_test(P)

[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.5 0.  0.  0.5]
 [0.  0.  0.5 0. ]]
[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  0.5]
 [0.5 0.  0.5 0. ]]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.  0.5]
 [0.5 0.  0.5 0. ]]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.5 0.  0.  0. ]]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.  0.5 0.  0. ]]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply


array([[ 1., inf, inf, inf, inf],
       [inf,  1., inf, inf, inf],
       [inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf]])

In [183]:
P0 = np.array([[1. , 0. , 0. , 0. , 0. ],
              [0. , 0 , 0. , 0. , 0. ],
              [0. , 0. , 0. , 0.5, 0.5],
              [0. , 0.5, 0. , 0. , 0.5],
              [0.5, 0. , 0. , 0.5, 0. ]])

In [511]:
p = np.array([[.5, .25, .25],[.5,0,.5],[.25,.25,.5]])
p

array([[0.5 , 0.25, 0.25],
       [0.5 , 0.  , 0.5 ],
       [0.25, 0.25, 0.5 ]])

In [512]:
fmpt(p)

array([[2.5       , 4.        , 3.33333333],
       [2.66666667, 5.        , 2.66666667],
       [3.33333333, 4.        , 2.5       ]])

In [513]:
fmpt_test(p)

array([[2.5       , 4.        , 3.33333333],
       [2.66666667, 5.        , 2.66666667],
       [3.33333333, 4.        , 2.5       ]])

In [520]:
p = np.array([[.5, .5, 0],[.9,0,.1],[.25,.25,.5]])
p

array([[0.5 , 0.5 , 0.  ],
       [0.9 , 0.  , 0.1 ],
       [0.25, 0.25, 0.5 ]])

In [521]:
fmpt_test(p)

[[0.   0.1 ]
 [0.25 0.5 ]]
[[0.5  0.  ]
 [0.25 0.5 ]]
[[0.5 0.5]
 [0.9 0. ]]


array([[ 1.63157895,  2.        , 30.        ],
       [ 1.26315789,  3.1       , 28.        ],
       [ 2.63157895,  3.        , 15.5       ]])

In [522]:
fmpt(p)

array([[ 1.63157895,  2.        , 30.        ],
       [ 1.26315789,  3.1       , 28.        ],
       [ 2.63157895,  3.        , 15.5       ]])

In [523]:
fmpt_test(p)

[[0.   0.1 ]
 [0.25 0.5 ]]
[[0.5  0.  ]
 [0.25 0.5 ]]
[[0.5 0.5]
 [0.9 0. ]]


array([[ 1.63157895,  2.        , 30.        ],
       [ 1.26315789,  3.1       , 28.        ],
       [ 2.63157895,  3.        , 15.5       ]])

In [524]:
fmpt(P)

array([[1.        ,        inf,        inf,        inf,        inf],
       [       inf, 1.        ,        inf,        inf,        inf],
       [       inf,        inf, 1.        ,        inf,        inf],
       [       inf,        inf,        inf, 2.        , 1.33333333],
       [       inf,        inf,        inf, 1.33333333, 2.        ]])

In [248]:
fmpt(P)

array([[1.        ,        inf,        inf,        inf,        inf],
       [       inf, 1.        ,        inf,        inf,        inf],
       [       inf,        inf, 1.        ,        inf,        inf],
       [       inf,        inf,        inf, 2.        , 1.33333333],
       [       inf,        inf,        inf, 1.33333333, 2.        ]])

In [197]:
fmpt(P)

array([[1.        ,        inf,        inf,        inf,        inf],
       [       inf, 1.        ,        inf,        inf,        inf],
       [       inf,        inf, 1.        ,        inf,        inf],
       [       inf,        inf,        inf, 2.        , 1.33333333],
       [       inf,        inf,        inf, 1.33333333, 2.        ]])

In [225]:
fmpt_test(P)

[inf  3.  2.  2.] [1.79769313e+308 3.00000000e+000 2.00000000e+000 2.00000000e+000]
[inf  3.  2.  2.] [1.79769313e+308 3.00000000e+000 2.00000000e+000 2.00000000e+000]
[inf inf  2.  2.] [1.79769313e+308 1.79769313e+308 2.00000000e+000 2.00000000e+000]
[inf inf 1.5 1. ] [1.79769313e+308 1.79769313e+308 1.50000000e+000 1.00000000e+000]
[inf inf 1.5 1. ] [1.79769313e+308 1.79769313e+308 1.50000000e+000 1.00000000e+000]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in multiply


array([[1. , inf, inf, inf, inf],
       [inf, 1. , inf, inf, inf],
       [3. , 3. , 3. , 1.5, 1.5],
       [2. , 2. , 2. , inf, 1. ],
       [2. , 2. , 2. , 1. , inf]])

In [211]:
fmpt_test(P)

/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in multiply


array([[1. , inf, inf, inf, inf],
       [inf, 1. , inf, inf, inf],
       [3. , 3. , 3. , 1.5, 1.5],
       [2. , 2. , 2. , inf, 1. ],
       [2. , 2. , 2. , 1. , inf]])

In [43]:
np.zeros_like(p)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [207]:
anp.nan

array([False, False])

In [206]:
np.where(a==np.nan, 0, a) 

array([ 1., nan])

In [201]:
np.nan_to_num?

In [213]:
 np.nan_to_num?

In [308]:
p = P
k = p.shape[0]
desti = 2
b = np.ones(k-1)
p_sub = np.delete(np.delete(p, desti, 0), desti,1)
p_calc = np.eye(k-1) - p_sub
m = np.full(k-1, np.inf)
row0 = (p_calc!=0).sum(axis=1)
none0 = np.arange(k-1)

try:
    m[none0] = np.linalg.solve(p_calc, b) 
except np.linalg.LinAlgError as err:
    if 'Singular matrix' in str(err):

        if (row0==0).sum() > 0:
            index0 = set(np.argwhere(row0 == 0).flatten())
            index0 = index0.union(set(np.argwhere(p_calc[:,list(index0)].sum(axis=1) != 0).flatten()))
            none0 = np.asarray(list(set(none0).difference(index0)))
            if len(none0) > 1:  
                p_calc = p_calc[none0,:][:, none0]
                b = b[none0]  
                m[none0] = np.linalg.solve(p_calc, b) 

[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.  0.5]
 [0.5 0.  0.5 0. ]] [[ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.  -0.5  1.  -0.5]
 [-0.5  0.  -0.5  1. ]]
{0, 1, 2, 3}


In [292]:
index0 = set(np.argwhere(row0 == 0).flatten())
none0 = np.asarray(list(set(none0).difference(index0)))
none0

array([2, 3])

In [289]:
p_calc_long = p_calc[none0,:]
p_calc_long

array([[ 0. , -0.5,  1. , -0.5],
       [-0.5,  0. , -0.5,  1. ]])

In [286]:
p_calc

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  1. , -0.5],
       [-0.5,  0. , -0.5,  1. ]])

In [295]:
p_calc[none0,:][:,list(index0)]

array([[ 0. , -0.5],
       [-0.5,  0. ]])

In [302]:
np.argwhere(p_calc[:,list(index0)].sum(axis=1) != 0 ).flatten()

array([2, 3])

In [293]:
p_calc[none0,:][:,list(index0)].sum(axis=1) == 0 

array([False, False])

In [280]:
p_calc_long[:,list(index0)].sum(axis=1) == 0 

array([-0.5, -0.5])

In [285]:
np.array([[1,2,3], [1,2,3]]).sum(axis=1)

array([6, 6])

In [267]:


for i in index0:
    if 

{0, 1}

In [413]:
m.p

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.88888889, 0.11111111, 0.        , 0.        ],
       [0.        , 0.09090909, 0.81818182, 0.09090909, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.9       ]])

In [501]:
m.p

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.88888889, 0.11111111, 0.        , 0.        ],
       [0.        , 0.09090909, 0.81818182, 0.09090909, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.9       ]])

In [503]:
fmpt_test(m.p)

/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in multiply


array([[ 1., inf, inf, inf, inf],
       [inf, inf,  9., 29., inf],
       [inf, inf, inf, 20., inf],
       [inf, inf, inf,  1., inf],
       [inf, inf, inf, 10., inf]])

In [462]:
p_calc = np.array([[ 0.    ,      0.   ,       0.     ,     0.        ],
 [ 0.     ,     0.18181818 ,-0.09090909,  0.       ],
 [ 0.   ,       0.    ,      0.       ,   0.        ],
 [ 0.   ,       0.   ,      -0.1    ,     0.1       ]])
p_calc

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.18181818, -0.09090909,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.1       ,  0.1       ]])

In [459]:
index0 = [0]

In [464]:
index0 = set(np.argwhere(row0 == 0).flatten())
m = np.full(k-1, np.inf)
row0 = (p_calc!=0).sum(axis=1)
none0 = np.arange(k-1)
print(index0)

{0, 2}


In [452]:
p_calc[:,list(index0)].sum(axis=1)

array([0., 0., 0., 0.])

In [454]:
m.p

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.88888889, 0.11111111, 0.        , 0.        ],
       [0.        , 0.09090909, 0.81818182, 0.09090909, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.9       ]])

In [497]:
f = ps.io.open(ps.examples.get_path('usjoin.csv'))
pci = np.array([f.by_col[str(y)] for y in range(1929, 2010)])
q5 = np.array([mc.Quantiles(y).yb for y in pci]).transpose()
m = Markov(q5[:, :2])

The Markov Chain is reducible and is composed by:
2 Recurrent classes (indices):
[0], [3]
2 Transient classes (indices):
[1 2], [4]
The Markov Chain has 2 absorbing states:
0, 3


In [498]:
fmpt_test(m.p)

[[0.88888889 0.11111111 0.         0.        ]
 [0.09090909 0.81818182 0.09090909 0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.1        0.9       ]]
[[ 0.11111111 -0.11111111  0.          0.        ]
 [-0.09090909  0.18181818 -0.09090909  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.1         0.1       ]]
{2}
{1, 2, 3} {0, 1, 3}
{0, 1, 2, 3} {0, 1, 3}

[[1.         0.         0.         0.        ]
 [0.         0.81818182 0.09090909 0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.1        0.9       ]]
[[ 0.          0.          0.          0.        ]
 [ 0.          0.18181818 -0.09090909  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.1         0.1       ]]
{0, 2}
{0, 1, 2, 3} {1, 3}

[[1.         0.         0.         0.        ]
 [0.         0.88888889 0.         0.        ]
 [0.         0.         1.         0.     

/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in multiply


array([[ 1., inf, inf, inf, inf],
       [inf, inf,  9., 29., inf],
       [inf, inf, inf, 20., inf],
       [inf, inf, inf,  1., inf],
       [inf, inf, inf, 10., inf]])

In [499]:
m.p

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.88888889, 0.11111111, 0.        , 0.        ],
       [0.        , 0.09090909, 0.81818182, 0.09090909, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.9       ]])

In [453]:
fmpt_test(m.p)

[[0.88888889 0.11111111 0.         0.        ]
 [0.09090909 0.81818182 0.09090909 0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.1        0.9       ]]
[[ 0.11111111 -0.11111111  0.          0.        ]
 [-0.09090909  0.18181818 -0.09090909  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.1         0.1       ]]
{2}
{1, 2, 3}

[[1.         0.         0.         0.        ]
 [0.         0.81818182 0.09090909 0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.1        0.9       ]]
[[ 0.          0.          0.          0.        ]
 [ 0.          0.18181818 -0.09090909  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.1         0.1       ]]
{0, 2}
{0, 1, 2, 3}

[[1.         0.         0.         0.        ]
 [0.         0.88888889 0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.1        

/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in multiply


array([[ 1., inf, inf, inf, inf],
       [ 9., inf,  9., 29.,  9.],
       [inf, inf, inf, 20., inf],
       [inf, inf, inf,  1., inf],
       [inf, inf, inf, 10., inf]])

In [455]:
m.p

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.88888889, 0.11111111, 0.        , 0.        ],
       [0.        , 0.09090909, 0.81818182, 0.09090909, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.9       ]])

In [434]:
p = np.array([[.5, .5, 0],[.2,0.8,0],[0,0,1]])
p

array([[0.5, 0.5, 0. ],
       [0.2, 0.8, 0. ],
       [0. , 0. , 1. ]])

In [352]:
a == 0

array([[False, False, False],
       [False, False, False],
       [False, False, False]])

In [354]:
np.isclose?

In [357]:
a

array([[ 3.50000000e+00,  2.00000000e+00, -2.52201579e+16],
       [ 5.00000000e+00,  1.40000000e+00, -2.52201579e+16],
       [            inf,             inf,  1.00000000e+00]])

In [363]:
import math

In [365]:
math.isclose?

In [364]:
math.isclose(-2.52201579e+16,0)

False

In [366]:
np.isclose?

In [370]:
np.set_printoptions(suppress=True)

In [374]:
-2.52201579e+16  <-0.3

True

In [375]:
np.isinf(-2.52201579e+16)

False

In [368]:
np.isclose(-2.52201579e+16,0,rtol=1e-02, atol=1e-1)

False

In [362]:
np.isclose?

In [356]:
np.isclose(a,0, atol=1e-05)

array([[False, False, False],
       [False, False, False],
       [False, False, False]])

In [380]:
np.nan_to_num(np.inf)

1.7976931348623157e+308

In [381]:
np.where(a<0,np.inf ,a)

array([[3.5, 2. , inf],
       [5. , 1.4, inf],
       [inf, inf, 1. ]])

In [382]:
P=np.array([[.5, .25, .25],[.5,0,.5],[.25,.25,.5]])

In [387]:
P

array([[0.5 , 0.25, 0.25],
       [0.5 , 0.  , 0.5 ],
       [0.25, 0.25, 0.5 ]])

In [384]:
P = np.matrix(P)
A = P ** 1000
A

matrix([[0.4, 0.2, 0.4],
        [0.4, 0.2, 0.4],
        [0.4, 0.2, 0.4]])

In [395]:
P = np.matrix(P)
A = P ** 1000

I = np.identity(k)
Z = la.inv(I - P + A)
E = np.ones_like(Z)
D = np.diag(1. / np.diag(A))
Zdg = np.diag(np.diag(Z))
M = (I - Z + E * Zdg) * D
ZM = Z * M
ZMdg = np.diag(np.diag(ZM))
W = M * (2 * Zdg * D - I) + 2 * (ZM - E * ZMdg)
W

matrix([[11.83333333, 28.        , 18.        ],
        [13.33333333, 37.        , 13.33333333],
        [18.        , 28.        , 11.83333333]])

In [408]:
f = ps.io.open(ps.examples.get_path('usjoin.csv'))
pci = np.array([f.by_col[str(y)] for y in range(1929, 2010)])
q5 = np.array([mc.Quantiles(y).yb for y in pci]).transpose()

In [409]:
m = Markov(q5[:, :2])
m.p

The Markov Chain is reducible and is composed by:
2 Recurrent classes (indices):
[0], [3]
2 Transient classes (indices):
[1 2], [4]
The Markov Chain has 2 absorbing states:
0, 3


array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.88888889, 0.11111111, 0.        , 0.        ],
       [0.        , 0.09090909, 0.81818182, 0.09090909, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.9       ]])

In [412]:
fmpt_test(m.p)

/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in multiply


array([[ 1., inf, inf, inf, inf],
       [ 9., inf,  9., 29.,  9.],
       [inf, inf, inf, 20., inf],
       [inf, inf, inf,  1., inf],
       [inf, inf, inf, 10., inf]])

In [407]:
np.where(P>0 and P>2, 1,2)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [396]:
P = np.asarray(P)
A = giddy.ergodic.steady_state(P)
A = np.tile(A,(k,1))

In [402]:
1e+16  >100

True

In [397]:
Z = la.inv(I - P + A)
E = np.ones_like(Z)
D = np.diag(1. / np.diag(A))
Zdg = np.diag(np.diag(Z))
M = (I - Z + E.dot(Zdg)).dot(D)
ZM = Z.dot(M)
ZMdg = np.diag(np.diag(ZM))
W = M.dot(2 * Zdg.dot(D) - I) + 2 * (ZM - E.dot(ZMdg))
W

array([[11.83333333, 28.        , 18.        ],
       [13.33333333, 37.        , 13.33333333],
       [18.        , 28.        , 11.83333333]])

array([[0.4, 0.2, 0.4],
       [0.4, 0.2, 0.4],
       [0.4, 0.2, 0.4]])

In [386]:
P ** 1000

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [379]:
np.abs(a) => 

SyntaxError: invalid syntax (<ipython-input-379-b27b6ee8bd30>, line 1)

In [348]:
a = fmpt_test(p)

[[0.2 0. ]
 [0.  0. ]]
[ 5. inf]
[3.5 5.  inf]
[[0.5 0. ]
 [0.  0. ]]
[ 2. inf]
[2.  1.4 inf]
[[ 0.5 -0.5]
 [-0.2  0.2]]
[-2.52201579e+16 -2.52201579e+16]
[-2.52201579e+16 -2.52201579e+16  1.00000000e+00]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


In [349]:
a

array([[ 3.50000000e+00,  2.00000000e+00, -2.52201579e+16],
       [ 5.00000000e+00,  1.40000000e+00, -2.52201579e+16],
       [            inf,             inf,  1.00000000e+00]])

In [350]:
np.isinf(a)

array([[False, False, False],
       [False, False, False],
       [ True,  True, False]])

In [346]:
a[np.isinf(a)] = np.inf

In [347]:
a

array([[ 3.50000000e+00,  2.00000000e+00, -2.52201579e+16],
       [ 5.00000000e+00,  1.40000000e+00, -2.52201579e+16],
       [            inf,             inf,  1.00000000e+00]])

In [343]:
fmpt_test(p)

[[0.2 0. ]
 [0.  0. ]]
[ 5. inf]
[3.5 5.  inf]
[[0.5 0. ]
 [0.  0. ]]
[ 2. inf]
[2.  1.4 inf]
[[ 0.5 -0.5]
 [-0.2  0.2]]
[-2.52201579e+16 -2.52201579e+16]
[-2.52201579e+16 -2.52201579e+16  1.00000000e+00]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


array([[ 3.50000000e+00,  2.00000000e+00, -2.52201579e+16],
       [ 5.00000000e+00,  1.40000000e+00, -2.52201579e+16],
       [            inf,             inf,  1.00000000e+00]])

In [337]:
fmpt_test(p)

[ 5. inf]
[3.5 5.  inf]
[ 2. inf]
[2.  1.4 inf]
[-2.52201579e+16 -2.52201579e+16]
[-2.52201579e+16 -2.52201579e+16  1.00000000e+00]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in multiply


array([[ 3.50000000e+00,  2.00000000e+00, -2.52201579e+16],
       [ 5.00000000e+00,  1.40000000e+00, -2.52201579e+16],
       [            inf,             inf,  1.00000000e+00]])

In [324]:
P

array([[1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0. , 0. , 0.5],
       [0.5, 0. , 0. , 0.5, 0. ]])

In [325]:
fmpt_test(P)

/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply


array([[1. , inf, inf, inf, inf],
       [inf, 1. , inf, inf, inf],
       [1.5, 1.5, inf, 1. , 1. ],
       [inf, 1. , inf, inf, inf],
       [1. , inf, inf, inf, inf]])

In [312]:
fmpt(P)

array([[1.        ,        inf,        inf,        inf,        inf],
       [       inf, 1.        ,        inf,        inf,        inf],
       [       inf,        inf, 1.        ,        inf,        inf],
       [       inf,        inf,        inf, 2.        , 1.33333333],
       [       inf,        inf,        inf, 1.33333333, 2.        ]])

In [307]:
P

array([[1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0. , 0. , 0.5],
       [0.5, 0. , 0. , 0.5, 0. ]])

In [264]:
fmpt_test(P)

[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.5 0.  0.  0.5]
 [0.  0.  0.5 0. ]]
P_sub: [0. 0. 0. 0.]
m: [inf inf inf inf]
[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  0.5]
 [0.5 0.  0.5 0. ]]
P_sub: [0. 0. 0. 0.]
m: [inf inf inf inf]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.  0.5]
 [0.5 0.  0.5 0. ]]
P_sub: [0.  0.  0.5 0.5]
m: [inf inf inf inf]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.5 0.  0.  0. ]]
P_sub: [0.  0.5 0.  0.5]
m: [inf inf inf inf]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.  0.5 0.  0. ]]
P_sub: [0.5 0.  0.  0.5]
m: [inf inf inf inf]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in multiply


array([[ 1., inf, inf, inf, inf],
       [inf,  1., inf, inf, inf],
       [inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf]])

In [ ]:
fmpt()

In [258]:
desti = 2
p_sub = np.delete(np.delete(P, desti, 0), desti,1)
p_calc = np.eye(k-1) - p_sub

In [260]:
p_calc

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  1. , -0.5],
       [-0.5,  0. , -0.5,  1. ]])

In [261]:
p_sub

array([[1. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. ],
       [0. , 0.5, 0. , 0.5],
       [0.5, 0. , 0.5, 0. ]])

In [259]:
np.linalg.solve(p_calc, np.arange(4)) 

LinAlgError: Singular matrix

In [253]:
fmpt_test(P)

[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.5 0.  0.  0.5]
 [0.  0.  0.5 0. ]]
P_sub: [0. 0. 0. 0.]
m: [inf  3.  2.  2.]
[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  0.5]
 [0.5 0.  0.5 0. ]]
P_sub: [0. 0. 0. 0.]
m: [inf  3.  2.  2.]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.  0.5]
 [0.5 0.  0.5 0. ]]
P_sub: [0.  0.  0.5 0.5]
m: [inf inf  2.  2.]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.5 0.  0.  0. ]]
P_sub: [0.  0.5 0.  0.5]
m: [inf inf 1.5 1. ]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.  0.5 0.  0. ]]
P_sub: [0.5 0.  0.  0.5]
m: [inf inf 1.5 1. ]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in multiply


array([[1. , inf, inf, inf, inf],
       [inf, 1. , inf, inf, inf],
       [3. , 3. , 3. , 1.5, 1.5],
       [2. , 2. , 2. , inf, 1. ],
       [2. , 2. , 2. , 1. , inf]])

In [239]:
fmpt_test(P)

[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.5 0.  0.  0.5]
 [0.  0.  0.5 0. ]]
P_sub: [0. 0. 0. 0.]
m: [inf  3.  2.  2.]
[[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  0.5]
 [0.5 0.  0.5 0. ]]
P_sub: [0. 0. 0. 0.]
m: [inf  3.  2.  2.]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.  0.5]
 [0.5 0.  0.5 0. ]]
P_sub: [0.  0.  0.5 0.5]
m: [inf inf  2.  2.]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.5 0.  0.  0. ]]
P_sub: [0.  0.5 0.  0.5]
m: [inf inf 1.5 1. ]
[[1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0.5]
 [0.  0.5 0.  0. ]]
P_sub: [0.5 0.  0.  0.5]
m: [inf inf 1.5 1. ]


/Users/weikang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


array([[1. , inf, inf, inf, inf],
       [inf, 1. , inf, inf, inf],
       [3. , 3. , 3. , 1.5, 1.5],
       [2. , 2. , 2. , inf, 1. ],
       [2. , 2. , 2. , 1. , inf]])

In [193]:
fmpt(p)

array([[2.5       , 4.        , 3.33333333],
       [2.66666667, 5.        , 2.66666667],
       [3.33333333, 4.        , 2.5       ]])

In [157]:
np.pad?

In [152]:
fmpt(P)

array([[inf, inf, inf, inf, inf],
       [3. , 3. , inf, inf, inf],
       [2. , 2. , 2. , 1.5, 1.5],
       [2. , 2. , 2. , 1. , 1. ]])

In [153]:
P

array([[1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5],
       [0. , 0.5, 0. , 0. , 0.5],
       [0.5, 0. , 0. , 0.5, 0. ]])

In [132]:
p = P
pp = np.asarray(p)
k = p.shape[0]
# first mean passage time
b = np.ones(k-1)
fmpt_all = np.zeros((k-1,k))
# for desti in range(k):
desti = 0
p_sub = np.delete(np.delete(p, desti, 0), desti,1)
p_calc = np.eye(k-1) - p_sub

print(desti, p_sub, p_calc) 
m = np.linalg.solve(p_calc, b)
fmpt_all[:, desti] = m

0 [[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.5 0.  0.  0.5]
 [0.  0.  0.5 0. ]] [[ 0.   0.   0.   0. ]
 [ 0.   1.  -0.5 -0.5]
 [-0.5  0.   1.  -0.5]
 [ 0.   0.  -0.5  1. ]]


LinAlgError: Singular matrix

In [133]:
np.linalg.solve(p_calc, b)

LinAlgError: Singular matrix

In [134]:
p_sub

array([[1. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5],
       [0.5, 0. , 0. , 0.5],
       [0. , 0. , 0.5, 0. ]])

In [135]:
p_calc

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5, -0.5],
       [-0.5,  0. ,  1. , -0.5],
       [ 0. ,  0. , -0.5,  1. ]])

In [136]:
[0,0,0,1].index(0)

0

In [137]:
list((p_calc!=0).sum(axis=1) == 0).index(0)

1

In [138]:
np.argwhere((p_calc!=0).sum(axis=1) == 0)[0].flatten()

array([0])

In [139]:
((p_calc!=0).sum(axis=1)==0).sum()

1

In [140]:
np.argwhere(np.array([0,0,0,1]) == 0).flatten()

array([0, 1, 2])

In [141]:
((p_calc!=0).sum(axis=1)==0)

array([ True, False, False, False])

In [142]:
row0 = (p_calc!=0).sum(axis=1)
if (row0==0).sum() > 0:
    index0 = set(np.argwhere(row0 == 0).flatten())
    none0 = np.asarray(list(set(range(k-1)).difference(index0)))

In [143]:
p_calc = p_calc[none0,:][:, none0]

In [144]:
p_calc

array([[ 1. , -0.5, -0.5],
       [ 0. ,  1. , -0.5],
       [ 0. , -0.5,  1. ]])

In [145]:
b = b[none0]
b

array([1., 1., 1.])

In [146]:
np.linalg.solve(p_calc, b)

array([3., 2., 2.])

In [123]:
b.reshape(-1,1)

array([[1.],
       [1.],
       [1.]])

In [104]:
set(range(k-1))

{0, 1, 2, 3}

In [105]:
index0

{0}

In [106]:
none0

array([1, 2, 3])

In [60]:
fmpt(p)

0 [[0.   0.5 ]
 [0.25 0.5 ]] [[ 1.   -0.5 ]
 [-0.25  0.5 ]]
1 [[0.5  0.25]
 [0.25 0.5 ]] [[ 0.5  -0.25]
 [-0.25  0.5 ]]
2 [[0.5  0.25]
 [0.5  0.  ]] [[ 0.5  -0.25]
 [-0.5   1.  ]]


array([[2.66666667, 4.        , 3.33333333],
       [3.33333333, 4.        , 2.66666667]])

In [63]:
P.sum(axis=1)

array([1., 1., 1., 1., 1.])

In [62]:
fmpt(P)

0 [[1.  0.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.5 0.  0.  0.5]
 [0.  0.  0.5 0. ]] [[ 0.   0.   0.   0. ]
 [ 0.   1.  -0.5 -0.5]
 [-0.5  0.   1.  -0.5]
 [ 0.   0.  -0.5  1. ]]


LinAlgError: Singular matrix

In [29]:
k = p.shape[0]

In [26]:
d = 0
p_sub = np.delete(np.delete(p, d, 0), d,1)
p_sub

array([[0.  , 0.5 ],
       [0.25, 0.5 ]])

In [31]:
p_calc = np.eye(k-1) - p_sub
p_calc

array([[ 1.  , -0.5 ],
       [-0.25,  0.5 ]])

In [39]:
b = np.ones(k-1)
b

array([1., 1.])

In [40]:
m = np.linalg.solve(p_calc, b)
m

array([2.66666667, 3.33333333])

In [5]:
steady_state(P0)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [6]:
steady_state(P, )

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [7]:
from sympy import *

In [8]:
x, y, z = symbols('x y z')

In [9]:
Eq(x, y)


Eq(x, y)

In [10]:
linsolve?

In [11]:
A = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 10]])

In [11]:
fmpt(P0)

array([[1.        , 1.        , 1.        , 1.33333333, 1.33333333],
       [2.        , 1.        , 1.        , 1.33333333, 1.33333333],
       [4.        , 0.5       , 1.        , 0.33333333, 0.33333333],
       [3.33333333, 0.33333333, 1.        , 1.        , 0.66666667],
       [2.66666667, 0.66666667, 1.        , 0.66666667, 1.        ]])

In [20]:
import libpysal as ps
f = ps.io.open(ps.examples.get_path('usjoin.csv'))
pci = np.array([f.by_col[str(y)] for y in range(1929, 2010)])
q5 = np.array([mc.Quantiles(y).yb for y in pci]).transpose()
m = Markov(q5)

The Markov Chain is irreducible and is composed by:
1 Recurrent class (indices):
[0 1 2 3 4]
0 Transient class.
The Markov Chain has 0 absorbing state.


In [26]:
m = Markov(q5[:, :2])

The Markov Chain is reducible and is composed by:
2 Recurrent classes (indices):
[0], [3]
2 Transient classes (indices):
[1 2], [4]
The Markov Chain has 2 absorbing states:
0, 3


In [27]:
m.p

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.88888889, 0.11111111, 0.        , 0.        ],
       [0.        , 0.09090909, 0.81818182, 0.09090909, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.9       ]])

In [28]:
m.fmpt

array([[1.        ,        inf,        inf,        inf,        inf],
       [       inf, 1.78305684, 8.65447755,        inf,        inf],
       [       inf, 7.393034  , 2.2770465 ,        inf,        inf],
       [       inf,        inf,        inf, 1.        ,        inf],
       [       inf,        inf,        inf,        inf, 1.        ]])

In [29]:
(m.p.sum(axis=1) == 0).sum()

0

In [30]:
import quantecon as qe

In [32]:
mc = qe.MarkovChain(m.p)
mc

Markov chain with transition matrix 
P = 
[[1.         0.         0.         0.         0.        ]
 [0.         0.88888889 0.11111111 0.         0.        ]
 [0.         0.09090909 0.81818182 0.09090909 0.        ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.1        0.9       ]]

In [3]:
f = ps.io.open(ps.examples.get_path("usjoin.csv"))
pci = np.array([f.by_col[str(y)] for y in range(1929,2010)]).transpose()
m = FullRank_Markov(pci)

In [6]:
m.sojourn_time[:5]

array([5.71428571, 2.75862069, 2.22222222, 1.77777778, 1.66666667])

In [7]:
m.p[0, :5]

array([0.825 , 0.0625, 0.0625, 0.025 , 0.025 ])

In [8]:
m.fmpt[0, :5]

array([48.        , 87.96280048, 68.1089084 , 58.83306575, 41.77250827])